# 12 - Using View Structures:


### In this tutorial we will walk through many examples of how and what to use view structures for, and is meant as a hightlight of this function's ability.

If you do not see a potential capability please let the developers know of your desire on github!!


In [ ]:
from architector import (view_structures,
                         build_complex,
                         convert_io_molecule)
from architector.io_calc import CalcExecutor # Run XTB calculations
from architector.io_samplers import random_sampler # Create distortions
from architector.vibrations_free_energy import vibration_analysis # Vibrational analysis to get normal modes
import architector.arch_context_manage as arch_context_manage # Temporary directory generation
from ase.vibrations import Vibrations # ASE implemenation of vibrational analysis
import numpy as np 

In [ ]:
# Even before running architector, view_structures can be used to visualize SMILES strings!
# In the background, openbabel is used to build these in 3D
view_structures(['C','CN','COC','CC=CC'])

In [ ]:
# To start from Architector, let's build complexes to visualize
inp = {'core':{'metal':'Fe','coreCN':6},
       'ligands':['bipy']*2+['chloride']+['water'],
       'parameters':{'skip_duplicate_tests':True,
                     'assemble_method':'UFF',
                     'full_method':'UFF',
                     'relax':True}}
out = build_complex(inp)

In [ ]:
# At it's base, view structures can be called on any number of types of structures.
# On Architector output dictionaries (e.g. out) it will pull out the mol2string fields to visualize.
# These are sorted by energy by default
view_structures(out)

In [ ]:
# You can play with the size/shape of viewer grids:
view_structures(out,
                columns=2, # Check the number of columns the viewer will use.
                w=400, # Change the size of each viewer width in pixels. Default is 200
                h=400 # Change the size of each viewer height in pixels. Default is 200
                )
# This will thus make a 2X2 grid with larger versions of the molecules viewed.

In [ ]:
# Now, we will pull out just the mol2strings to highlight other view_structures functionality
mol_list = [val['mol2string'] for key,val in out.items()]
energies = [val['energy'] for key,val in out.items()] # Pull out the energies
energy_labels = ['{0:.2f}'.format(x) for x in (np.array(energies)-min(energies))] # Format and make the energies relative to minimum energy

In [ ]:
# We can now add the energies of the structures as labels.
view_structures(mol_list,
                labels=energy_labels # Add labels to the centroid of the molecules with the relative energies
                )

In [ ]:
# You can also add arbitrary strings or numbers as labels
view_structures(mol_list,
                labels=np.arange(len(mol_list)) # Add labels corresponding to the indices of the structures
                )
view_structures(mol_list,
                labels=['zero','one','two',
                        'three','four','five'][:len(mol_list)] # Add labels up to the number of structures produced
                )

In [ ]:
# Interatomic distances can also be visualized
view_structures(mol_list,
                vis_distances=True)
# For much more parameters on how to edit and tweak visualization of 
# interatomic distances please see tutorial 11-Distance_Analysis.ipynb!

In [ ]:
# Similar to labels, you can request to label the indices in each structure.
view_structures(mol_list,
                w=400, # Make the viewer larger
                h=400, # Make the viewer larger
                columns=2, # Switch to two columns
                labelinds=True, # Label the indices of the atoms in the molecule
                )

In [ ]:
# For simplicity, we can select a single structure to view the indices of the atoms in the molecule
mol = convert_io_molecule(mol_list[1])
# Similar to labels, you can pass a list of values or strings to visualize on top of the indices.
view_structures(mol,
                w=400, # Make the viewer larger
                h=400, # Make the viewer larger
                labelinds=(np.arange(len(mol.ase_atoms))/10).tolist(), # Label the indices of the atoms in the molecule
                )
# Note that this is useful for visualizing charges
view_structures(mol,
                w=400, # Make the viewer larger
                h=400, # Make the viewer larger
                labelinds=mol.ase_atoms.get_chemical_symbols(), # Add chemical symbols
                )
labelinds = list(map(chr, range(97, 123))) * 3
view_structures(mol,
                w=400, # Make the viewer larger
                h=400, # Make the viewer larger
                labelinds=labelinds[:len(mol.ase_atoms)], # Add arbitrary letters to each atom
                )

In [ ]:
# Note, this can all be done for lists of structures as well!!!
labelinds = list(map(chr, range(97, 123))) * 3
view_structures(mol_list,
                labelinds=[labelinds[:len(mol.ase_atoms)]]*len(mol_list), # Add arbitrary letters to each atom for each structure in list
                )

In [ ]:
# Now, we can also play with the parameters for the visualization itself.
view_structures(mol,
                representation='stick', # Switch to stick representation
                stick_scale=0.5, # Make the sticks larger
                )
view_structures(mol,
                representation='sphere' # Switch to sphere representation
                )
view_structures(mol,
                sphere_scale=0.5, # Increase sphere scales
                metal_scale=1.3, # Increase metal sphere scale even more.
                )

In [ ]:
# Now, we can distort and relax a structure with XTB with in-built routines:
distortion,_,_ = random_sampler(mol,
                                n=1, # Produce one distortion
                                max_rmsd=0.2 # Make sure it's not too different 
                                ) 
distortion = distortion[0]
view_structures([mol,distortion],
                labels=['UFF Minima','Distorted'],
                columns=2)

In [ ]:
# Now, we can relax the distortion
xtb_relaxed = CalcExecutor(mol,
                           method='GFN2-xTB',
                           relax=True,
                           save_trajectories=True)

In [ ]:
# Now we can visulaze both the distortion and the relaxed structures
view_structures([distortion,xtb_relaxed.mol],
                labels=['Distorted','XTB Relaxed'],
                columns=2
                )

In [ ]:
# Further, we can animate the relaxation trajectory! 
view_structures(xtb_relaxed.trajectory,
                w=400,
                h=400,
                trajectory=True # Treat as a trajectory for animation
                )
# Note that anything that is a list of structures that architector can read (ase atoms, xyz, mol2 ...)
# Can be treated as a trajectory

In [ ]:
# Further, we can animate the relaxation trajectory! 
view_structures(xtb_relaxed.trajectory,
                w=400,
                h=400,
                interval=400, # Slow down the trajectory animation
                trajectory=True)

In [ ]:
# Note that this block takes around 1 minute on an M2Max Macbook Pro.
calced = CalcExecutor(xtb_relaxed.mol,
                      method='GFN2-xTB') # Re-run single point to get vibrations
with arch_context_manage.make_temp_directory() as _:
    vib_analysis = Vibrations(calced.mol.ase_atoms) # Run vibrational analysis using ASE
    vib_analysis.run()
    data = vib_analysis.get_vibrations() # Get the vibrations
    hess = data.get_hessian_2d() # Get the hessian 
# Calculate the normal modes, force constnaces, and vibrational energies.
vib_energies, modes, fconstants, _ , frequencies = vibration_analysis(calced.mol.ase_atoms,
                                                                      hess)

In [ ]:
# With the vibrational modes, we can now visualize mode vibrations
view_structures(calced.mol,
                modes=[modes[7]] # Visualize just one mode (non-zero modes start a 7)
                )

In [ ]:
# We can also visualize more than one mode
view_structures([calced.mol]*8,
                modes=modes[30:(30+8)], # Visualize First 8 modes
                labels=frequencies[30:(30+8)] # Add labels corresponding to the frequencies modes (cm^-1)
                )

# Conclusions to using View Structures

In this tutorial we walked through several examples highlighting view structures capabilites.

Although all of these examples were performed using Architector-generated structures,

All these capabilities can also be done with local files or paths pointing to structure files including xyz, mol2 ...